# Main Application Entry Point

> Main application integration and command-line interface for the new GzKeyboard system

In [ ]:
#| default_exp main

In [ ]:
#| export
__all__ = ['main']

import argparse
import sys
import time
from pathlib import Path

from .core import CharacterStore
from .composition import SyllableComposer, CompositionStrategy, test_composition_interactive
from .system import SystemInputHandler, InputMode

In [ ]:
#| export
def main():
    """Main command-line entry point."""
    parser = argparse.ArgumentParser(
        description="GzKeyboard v2.0 - Advanced Geez keyboard for Tigrinya and Amharic"
    )
    
    parser.add_argument(
        "--mode", 
        choices=["tigrinya", "amharic", "latin"],
        default="tigrinya",
        help="Input mode (default: tigrinya)"
    )
    
    parser.add_argument(
        "--test",
        action="store_true",
        help="Run interactive composition tests"
    )
    
    args = parser.parse_args()
    
    if args.test:
        print("🧪 GzKeyboard Interactive Tests")
        
        # Create test setup
        character_store = CharacterStore()
        composer = SyllableComposer(character_store, strategy=CompositionStrategy.ADAPTIVE)
        
        # Test common words
        test_words = ['hello', 'selam', 'geez']
        
        for word in test_words:
            output = test_composition_interactive(composer, word, show_details=False)
            print(f"   '{word}' → '{output}'")
        
        print("\\n✅ Tests completed!")
        return
    
    # Start the actual keyboard system
    print(f"🎹 Starting GzKeyboard v2.0 in {args.mode} mode...")
    
    try:
        # Create input mode enum
        input_mode = InputMode(args.mode.upper())
        
        # Create system handler with basic config
        from .system import KeyboardConfig
        config = KeyboardConfig(
            default_mode=input_mode,
            debug_mode=False
        )
        
        handler = SystemInputHandler(config)
        
        # Try to setup keyboard hooks
        success = handler.setup_keyboard_hook()
        
        if success:
            print(f"✅ Keyboard hooks active in {args.mode} mode")
            print(f"⌨️  Hotkeys:")
            print(f"   {config.toggle_hotkey}: Toggle on/off")
            print(f"   {config.tigrinya_hotkey}: Tigrinya mode")
            print(f"   {config.amharic_hotkey}: Amharic mode")
            print(f"   {config.latin_hotkey}: Latin mode")
            print(f"\\n✨ Start typing! Press Ctrl+C to exit.")
            
            # Keep running
            try:
                while True:
                    time.sleep(0.1)
            except KeyboardInterrupt:
                print("\\n👋 GzKeyboard stopped. Goodbye!")
                handler.cleanup()
        else:
            print("❌ Failed to start keyboard hooks.")
            print("💡 Try running with --test to verify the composition engine works")
    
    except Exception as e:
        print(f"❌ Error: {e}")
        print("💡 Try running with --test to test the composition engine")

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()